In [1]:
import qubx

%qubxd

%load_ext autoreload
%autoreload 2

from typing import cast
import numpy as np
import pandas as pd
import asyncio
from pathlib import Path
from qubx.core.lookups import lookup
from qubx.connectors.ccxt.factory import get_ccxt_exchange
from qubx import logger, QubxLogConfig
from qubx.core.interfaces import IStrategy, IStrategyContext, BaseErrorEvent, IStrategyInitializer
from qubx.utils.runner.runner import run_strategy, StrategyConfig, AccountConfigurationManager, ExchangeConfig, LoggingConfig
from qubx.utils.runner.configs import LiveConfig, ReaderConfig
from qubx.core.basics import DataType, MarketEvent
from qubx.utils.charting.lookinglass import LookingGlass

QubxLogConfig.set_log_level("INFO")


⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2025, ver. 0.6.91
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


## 1.0 Define simple strategy that does nothing

Just subscribes to OHLC[1h], orderbook aggregated in 0.01% buckets top 100 levels (1% depth).

In [2]:
class TestStrategy(IStrategy):

    def on_init(self, initializer: IStrategyInitializer):
        initializer.set_base_subscription(DataType.TRADE)
        initializer.set_subscription_warmup({
            DataType.OHLC["1h"]: "1d"
        })
    
    def on_market_data(self, ctx: IStrategyContext, data: MarketEvent):
        pass

    def on_error(self, ctx: IStrategyContext, error: BaseErrorEvent) -> None:
        logger.error(f"Error: {error}")

    def on_stop(self, ctx: IStrategyContext):
        logger.info("Stopping test strategy")

In [ ]:
ctx = run_strategy(
    config=StrategyConfig(
        name="TestStrategy",
        strategy=TestStrategy,
        aux=ReaderConfig(reader="xlighter", args={"max_history": "10d"}),
        live=LiveConfig(
            exchanges={
                "LIGHTER": ExchangeConfig(
                    connector="xlighter",
                    universe=["BTCUSDC", "ETHUSDC"],
                )
            },
            logging=LoggingConfig(
                logger="InMemoryLogsWriter",
                position_interval="10s",
                portfolio_interval="1m",
                heartbeat_interval="10m",
            )
        )
    ),
    account_manager=AccountConfigurationManager(search_qubx_dir=True),
    paper=True,
    blocking=False,
)

DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector


2025-10-10 10:59:17.331 [ℹ️] Initialized LighterClient (testnet=False, account_index=0, api_key_index=0)
2025-10-10 10:59:17.332 [ℹ️] Loading instruments from Lighter API...
2025-10-10 10:59:18.616 [ℹ️] Found 92 markets
2025-10-10 10:59:18.618 [ℹ️] Successfully loaded 92 instruments
2025-10-10 10:59:18.620 [ℹ️] XLighterDataReader initialized: 92 instruments loaded, max_history=10d


DEBUG:asyncio:Using selector: EpollSelector


2025-10-10 10:59:18.742 [ℹ️] Initialized LighterClient (testnet=False, account_index=0, api_key_index=0)
2025-10-10 10:59:18.744 [ℹ️] Loading instruments from Lighter API...
2025-10-10 10:59:20.003 [ℹ️] Found 92 markets
2025-10-10 10:59:20.005 [ℹ️] Successfully loaded 92 instruments
2025-10-10 10:59:20.008 [ℹ️] LighterDataProvider initialized
2025-10-10 10:59:20.011 [ℹ️] - Strategy: TestStrategy
- Mode: paper
- Parameters: {}
2025-10-10 10:59:20.039 [ℹ️] Starting warmup for 2 configurations
2025-10-10 10:59:20.040 [⚠️] qubx.connectors.xlighter.data:_warmup_instrument:446 - Warmup not supported for ohlc
2025-10-10 10:59:20.042 [⚠️] qubx.connectors.xlighter.data:_warmup_instrument:446 - Warmup not supported for ohlc
2025-10-10 10:59:20.044 [ℹ️] Warmup complete
2025-10-10 10:59:20.050 [ℹ️] WebSocket connection initiated
2025-10-10 10:59:20.052 [ℹ️] Connecting to wss://mainnet.zklighter.elliot.ai/stream
2025-10-10 10:59:20.059 [ℹ️] Subscribed to trade for 2 instruments: ['ETHUSDC', 'BTCUSD

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=2)
           2	RESUME(arg=0, lineno=2)
           4	PUSH_NULL(arg=None, lineno=3)
           6	LOAD_FAST(arg=0, lineno=3)
           8	LOAD_ATTR(arg=0, lineno=3)
          28	LOAD_FAST(arg=1, lineno=3)
          30	CALL_FUNCTION_EX(arg=0, lineno=3)
          32	RETURN_VALUE(arg=None, lineno=3)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc_initial: State(pc_initial=0 nstack_initial=0)
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=2)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=RESUME(arg=0, lineno=2)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=PUSH_NULL(arg=None, lineno=3)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_FAST(arg=0, lineno=3)
DEBUG:numba.core.byteflow:stack ['$nu

...

2025-10-10 10:59:22.688 [ℹ️] Waiting for all instruments (1/2 ready). Missing: ['BTCUSDC']. Will start with partial data in 60s
2025-10-10 10:59:22.688 [ℹ️] Heartbeat at 2025-10-10T10:50:00
2025-10-10 10:59:22.700 [ℹ️] All 2 instruments have data - strategy ready to start


DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [738 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [4017 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [719 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [784 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [752 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [718 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:0","liquidation_trades":null,...,"type":"update/trade"}' [775 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:0","liquidation_trades":null,...,"type":"update/trade"}' [775 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"tra

2025-10-10 10:59:45.685 [ℹ️] Stopping test strategy
2025-10-10 10:59:45.686 [ℹ️] Disconnecting from WebSocket


DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [1406 bytes]
DEBUG:websockets.client:< TEXT '{"channel":"trade:1","liquidation_trades":null,...,"type":"update/trade"}' [1378 bytes]
DEBUG:websockets.client:< CLOSE 1000 (OK) [2 bytes]
DEBUG:websockets.client:< EOF
DEBUG:websockets.client:> EOF
DEBUG:websockets.client:= connection is CLOSED
DEBUG:websockets.client:x closing TCP connection


2025-10-10 10:59:45.951 [ℹ️] Disconnected from WebSocket
2025-10-10 10:59:45.952 [ℹ️] LighterDataProvider closed
2025-10-10 10:59:45.954 [ℹ️] [StrategyContext] :: Market data processing stopped


In [8]:
ctx.stop()

DEBUG:websockets.client:> CLOSE 1000 (OK) [2 bytes]
DEBUG:websockets.client:= connection is CLOSING


In [ ]:
add_symbols = ["BTCUSDC", "ETHUSDC", "XRPUSDC", "SOLUSDC", "DOGEUSDC", "ADAUSDC", "LINKUSDC"]
add_instruments = [ctx.query_instrument(symbol) for symbol in add_symbols]
print(add_instruments)
ctx.set_universe(add_instruments)

In [ ]:
ctx.ohlc(ctx.query_instrument("LINKUSDC"), "1h").pd().tail(2)

In [ ]:
ctx.subscribe(DataType.ORDERBOOK[0.0, 10])

In [ ]:
ohlc = ctx.ohlc(ctx.query_instrument("BTCUSD"), "1h").pd()

LookingGlass(
    ohlc,
    {
        "volume": ["bars", ohlc["volume"]],
    },
    backend="mpl",
).look()